# FAISS for UTF-8 Multihot Decoding

This notebook intends to learn how to use faiss and check if it is usefull to handle the decoding (cosine similarity ?) of embedding vectors into code indices of the codebook.


In [1]:
import numpy as np
import itertools
from itertools import combinations
import faiss
from utf8_encoder import *

Loading faiss with AVX2 support.
Loading faiss.


Load the pre-computed utf-8 codes and codebook


In [2]:
import torch
import torch.nn as nn
from langmodels.utf8codec import *
from langmodels import utf8codec

In [3]:
# utf8codebook = load_obj("utf8-codes/multihot64short-embeds")
# idx2char = load_obj("utf8-codes/multihot64short-idx2char")
# char2idx = load_obj("utf8-codes/multihot64short-char2idx")

# utf8codebook = load_obj("utf8-codes/utf8_code_matrix_2seg.npy")
utf8codebook = np.load("utf8-codes/utf8_code_matrix_2seg.npy").astype("float32")
idx2char = load_obj("utf8-codes/num2txt_2seg.pkl")
char2idx = load_obj("utf8-codes/txt2num_2seg.pkl")

In [4]:
code_matrix, txt2code, code2txt, txt2num, num2txt = utf8codec._load_codebook()

In [5]:
utf8codebook.dtype

dtype('float32')

In [6]:
# d=64
d=324
indexl2 = faiss.IndexFlatL2(d)  # the index is small for 2 segments dimension so a flat index that is completely in memory
# faiss.index_factory()  # <- this function for index creation

from the documentation, maybe will be faster for decoding to use an IVF (Inverse Vector File) index type

Also it might be good to have it in CPU as the GPU operations might not be needed for training or testing, the decoding can be done just when needed to create a text reconstruction for user visualization. Taking into account that I have much more RAM than GPU-RAM (a factor of 8) this would be a nice thing.

IVF indices need training

Product Quantization (PQ) could be used and might be good, this is because the input are binary elements, and the outputs of the network are float32. If quantized, the most important decimals of the vector output might be good enough to recognize similarity (just an idea, I don't know if this will be true).

Might need *OPQ rotation* and|or *RemapDimensionsTransform* (from documentation) to improve the PQ coding (transformations can be trained) and *rq = faiss.IndexRefineFlat(q)* to refine ranking once pre-ranked

From documentation:

    "The IVFFlat is often the fastest option, so the PQ variants are useful if memory is limited."



The other nice thing of this is that we can compute the closest K nearest neighbours, which can give several benefits for decoding.

I could try to do iterative decoding, several passes for the same input, each giving a result, this result passing it again, and again, this might lead to some nice surprises (estabilizing the result when there are doubts or different options??)

In [7]:
utf8codebook.shape

(1984, 324)

In [8]:
indexl2.add(utf8codebook)

Now what I do is encode all the codebook and decode it with the overfit-pre-trained Autoencoder

First load the pre-trained models

In [9]:
# utf8embedd = torch.load("trained_models/utf8Autoencoder_embedding_2segments_d48.pth")
# utf8decod = torch.load("trained_models/utf8Autoencoder_decoder_2segments_d48.pth")

utf8embedd = torch.load("trained_models/utf8Autoencoder_embedding_2segments_d64.pth")
utf8decod = torch.load("trained_models/utf8Autoencoder_decoder_2segments_d64.pth")

FileNotFoundError: [Errno 2] No such file or directory: 'trained_models/utf8Autoencoder_embedding_2segments_d64.pth'

Now I get the dataset and the codes for each elements to test and check all the codes and see the errors.

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
utf8embedd = utf8embedd.to(device)
utf8decod = utf8decod.to(device)

In [ ]:
# utf8embedd

In [ ]:
# utf8decod

In [ ]:
# prepare data to encode and decode
chars, idxs = list(char2idx.keys()), list(char2idx.values())

In [ ]:
# list(zip(chars,idxs))

In [ ]:
indices = np.stack(idxs)
tindices = torch.from_numpy(indices).to(device)

In [ ]:
# list(indices)[-100:]

In [ ]:
tindices[-30:]

In [ ]:
tindices = tindices.unsqueeze(0)
tindices.shape

In [ ]:
enc_multihot, enc32 = utf8embedd(tindices)

In [ ]:
enc_multihot.shape, enc32.shape


In [ ]:
dec = utf8decod(enc32)

In [ ]:
type(dec)

In [ ]:
bdec = dec > 0.

In [ ]:
bdec

In [ ]:
cmp = bdec.squeeze() == enc_multihot.squeeze()

In [ ]:
cmp.shape

In [ ]:
npdec = dec.squeeze().cpu().detach().numpy()

In [ ]:
npdec.dtype

In [ ]:
npdec.shape

In [ ]:
k=2
D,I = indexl2.search(npdec, k)

In [ ]:
# search by threshold range instead of K closests
indexl2.range_search?

In [ ]:
faiss.MatrixStats(npdec).comments

In [ ]:
type(I), I.dtype

In [ ]:
I.shape

In [ ]:
I0 = I[:,0] 
errvals = np.roll(I0,1) - I0

In [ ]:
errs = [i for i in errvals if abs(i) != 1]

In [ ]:
len(errs)

In [ ]:
I0.shape

In [ ]:
I0[-3:]

In [ ]:
for i in [2,3,15]:
    print(idx2char[i])

In [ ]:
res = []
for i in range(len(idx2char.keys())):
    o = idx2char[i]
    d = idx2char[I0[i]]
    if o != d:
        res.append((i,o,d))
    
    

In [ ]:
len(res)

In [ ]:
res


### For encoding dimension 32

There are 102 errors in the decoding on the first choice, anything more than zero is a problem so I have to go back and try to train a new encoder with maybe more dimensions to see what happens.

Anyways I stil have to find a way to make a ZERO error on the overfitted model

### For encoding dimension 48

For dimension 48 there are only 7 errors:

    [(17, '\x11', '>'),
     (24, '\x18', '>'),
     (33, '!', '>'),
     (35, '#', '>'),
     (41, ')', '\\'),
     (65, 'A', '>'),
     (90, 'Z', '>')]

### For encoding dimension 64

There are no errors at all for this dimensionality of the encoding and with only 8000 batches (125 epochs, 22 minutes training)  ;)  YAY

In [ ]:
I,D

In [ ]:
torch.sum(bdec), torch.sum(~cmp), torch.sum(enc_multihot), np.sum(utf8codebook)

For the encoding of dimension 32 the $ sum ( cmp )== 102 $  in the comparison and the different sum in the codebook and the enc_multihot seem to show some mismatch, which I have to check

In [ ]:
bdec.shape

In [ ]:
enc_multihot.shape

In [ ]:
enc_multihot.shape, enc32.shape, dec.shape

In [ ]:
# what I should do is create several indices and test the decoding in all of those
# for performance to see which one is better in precision, memory usage and speed

Now that I have the encoding and decoding working, What I do is I save the new encoding (which is the result ofpassing the entire domain through the overfitted encoder) to disk, this will be now the new "overfitted" embedding of dimension 64 

In [ ]:
embedding_matrix = enc32.detach().cpu().numpy()

In [ ]:
np.save("utf8-codes/utf8_codebook_overfit_matrix_2seg_dim64.npy", embedding_matrix)

In [ ]:
# # using the GPU
# res = faiss.StandardGpuResources()  # use a single GPU
# gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, indexl2)


In [ ]:
# print(gpu_index_flat.ntotal)


The index seems quite big here for the encoding of the complete utf-8 4 segments (+1M datapoints)
Mem usage of the GPU is 3028 MiB with the index in GPU

Mem usage of the GPU without the index: 1573MiB

So total memory usage of the index is: 1455 MiB for the IndexFlatL2 on the entire utf8 codebook coded in vectors of dim 64 


In [ ]:
# 3028 - 1573